# Introduction to PyTorch

## A.1 What is PyTorch

In [15]:
import torch

print(torch.__version__)

2.5.1+cu121


In [16]:
print(torch.cuda.is_available())

True


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/appendix-a_compressed/1.webp" width="400px">

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/appendix-a_compressed/2.webp" width="300px">

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/appendix-a_compressed/3.webp" width="300px">

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/appendix-a_compressed/4.webp" width="500px">

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/appendix-a_compressed/5.webp" width="500px">

## A.2 Understanding tensors

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/appendix-a_compressed/6.webp" width="400px">

### A.2.1 Scalars, vectors, matrices, and tensors

In [17]:
import torch
import numpy as np

# create a 0D tensor (scalar) from a Python integer
tensor0d = torch.tensor(1)

# create a 1D tensor (vector) from a Python list
tensor1d = torch.tensor([1, 2, 3])

# create a 2D tensor from a nested Python list
tensor2d = torch.tensor([[1, 2], 
                         [3, 4]])

# create a 3D tensor from a nested Python list
tensor3d_1 = torch.tensor([[[1, 2], [3, 4]], 
                           [[5, 6], [7, 8]]])

# create a 3D tensor from NumPy array
ary3d = np.array([[[1, 2], [3, 4]], 
                  [[5, 6], [7, 8]]])
tensor3d_2 = torch.tensor(ary3d)  # Copies NumPy array
tensor3d_3 = torch.from_numpy(ary3d)  # Shares memory with NumPy array

In [18]:
ary3d[0, 0, 0] = 999
print(tensor3d_2) # remains unchanged

tensor([[[1, 2],
         [3, 4]],

        [[5, 6],
         [7, 8]]])


In [19]:
print(tensor3d_3) # changes because of memory sharing

tensor([[[999,   2],
         [  3,   4]],

        [[  5,   6],
         [  7,   8]]])


### A.2.2 Tensor data types

In [20]:
tensor1d = torch.tensor([1, 2, 3])
print(tensor1d.dtype)

torch.int64


In [21]:
floatvec = torch.tensor([1.0, 2.0, 3.0])
print(floatvec.dtype)

torch.float32


In [22]:
floatvec = tensor1d.to(torch.float32)
print(floatvec.dtype)

torch.float32


### A.2.3 Common PyTorch tensor operations

In [23]:
tensor2d = torch.tensor([[1, 2, 3], 
                         [4, 5, 6]])
tensor2d

tensor([[1, 2, 3],
        [4, 5, 6]])

In [24]:
tensor2d.shape

torch.Size([2, 3])

*One of the most common errors in deep learning (shape errors)*

Because much of deep learning is multiplying and performing operations on matrices and matrices have a strict rule about what shapes and sizes can be combined, one of the most common errors you'll run into in deep learning is *shape mismatches*.

In [25]:
tensor2d.matmul(tensor2d)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x3 and 2x3)

We can make matrix multiplication work by making their inner dimensions match.

One of the ways to do this is with a transpose (switch the dimensions of a given tensor).

You can perform transposes in PyTorch using either:

- `torch.transpose(input, dim0, dim1)` - where input is the desired tensor to transpose and dim0 and dim1 are the dimensions to be swapped.
- `tensor.T` - where tensor is the desired tensor to transpose.

In [13]:
tensor2d.T

tensor([[1, 4],
        [2, 5],
        [3, 6]])

In [26]:
tensor2d.matmul(tensor2d.T)

tensor([[14, 32],
        [32, 77]])

In [15]:
tensor2d @ tensor2d.T

tensor([[14, 32],
        [32, 77]])

### A.2.4 Tensor Shapes and Dimensions


Often times you'll want to reshape or change the dimensions of your tensors without actually changing the values inside them.

To do so, some popular methods are:

| Method | One-line description |
| ----- | ----- |
| [`torch.reshape(input, shape)`](https://pytorch.org/docs/stable/generated/torch.reshape.html#torch.reshape) | Reshapes `input` to `shape` (if compatible), can also use `torch.Tensor.reshape()`. |
| [`Tensor.view(shape)`](https://pytorch.org/docs/stable/generated/torch.Tensor.view.html) | Returns a view of the original tensor in a different `shape` but shares the same data as the original tensor. |
| [`torch.squeeze(input)`](https://pytorch.org/docs/stable/generated/torch.squeeze.html) | Squeezes `input` to remove all the dimenions with value `1`. |
| [`torch.unsqueeze(input, dim)`](https://pytorch.org/docs/1.9.1/generated/torch.unsqueeze.html) | Returns `input` with a dimension value of `1` added at `dim`. | 
| [`torch.permute(input, dims)`](https://pytorch.org/docs/stable/generated/torch.permute.html) | Returns a *view* of the original `input` with its dimensions permuted (rearranged) to `dims`. | 

Why do any of these?

Because deep learning models (neural networks) are all about manipulating tensors in some way. And because of the rules of matrix multiplication, if you've got shape mismatches, you'll run into errors. These methods help you make sure the right elements of your tensors are mixing with the right elements of other tensors. 

In [27]:
x = torch.arange(1., 8.)

print("Original shape:", x.shape)

Original shape: torch.Size([7])


In [32]:
# Add an extra dimension
x_reshaped = x.reshape(1, 7)
x_reshaped, x_reshaped.shape

(tensor([[1., 2., 3., 4., 5., 6., 7.]]), torch.Size([1, 7]))

In [34]:
# Change view (keeps same data as original but changes view)
z = x.view(1, 7)
z, z.shape, x.shape

(tensor([[1., 2., 3., 4., 5., 6., 7.]]), torch.Size([1, 7]), torch.Size([7]))


Remember though, changing the view of a tensor with `torch.view()` really only creates a new view of the same tensor.

So changing the view changes the original tensor too.

In [35]:
# Changing z changes x
z[:, 0] = 5
z, x

(tensor([[5., 2., 3., 4., 5., 6., 7.]]), tensor([5., 2., 3., 4., 5., 6., 7.]))

How about removing all single dimensions from a tensor? To do so you can use `torch.squeeze()` 

In [31]:
# Remove dimension
x_squeezed = x_unsqueezed.squeeze()
print("After squeeze:", x_squeezed.shape)

After squeeze: torch.Size([7])


And to do the reverse of `torch.squeeze()` you can use torch.unsqueeze() to add a dimension value of 1 at a specific index.

In [30]:
# Add dimension
x_unsqueezed = x.unsqueeze(0)
print("After unsqueeze:", x_unsqueezed.shape)

After unsqueeze: torch.Size([1, 7])


You can also rearrange the order of axes values with `torch.permute(input, dims)`, where the input gets turned into a view with new dims.

In [37]:
# Create tensor with specific shape
x_original = torch.rand(size=(200, 100, 3))

# Permute the original tensor to rearrange the axis order
x_permuted = x_original.permute(2, 0, 1) # shifts axis 0->1, 1->2, 2->0

print(f"Previous shape: {x_original.shape}")
print(f"New shape: {x_permuted.shape}")

Previous shape: torch.Size([200, 100, 3])
New shape: torch.Size([3, 200, 100])


> Note: Because permuting returns a view (shares the same data as the original), the values in the permuted tensor will be the same as the original tensor and if you change the values in the view, it will change the values of the original.

### A.2.5 Tensor Aggregation

Calculating statistics like the mean or standard deviation can be useful for understanding your dataset or for implementing certain types of normalization during the data preparation phase.

In [39]:
x = torch.arange(0, 50, 5)
x

tensor([ 0,  5, 10, 15, 20, 25, 30, 35, 40, 45])

In [57]:
print(f"Minimum: {x.min()}")
print(f"Maximum: {x.max()}")
# print(f"Mean: {x.mean()}") # this will error
print(f"Mean: {x.type(torch.float32).mean()}") # won't work without float datatype
print(f"Std: {x.type(torch.float32).std()}") # won't work without float datatype
print(f"Sum: {x.sum()}")

Minimum: 1
Maximum: 9
Mean: 5.0
Std: 2.7386128902435303
Sum: 45


You can also do the same as above with `torch` methods:

In [56]:
torch.max(x), torch.min(x), torch.mean(x.type(torch.float32)), torch.sum(x), torch.std(x.float())

(tensor(9), tensor(1), tensor(5.), tensor(45), tensor(2.7386))

You can also find the index of a tensor where the max or minimum occurs with `torch.argmax()` and `torch.argmin()` respectively.

This is helpful incase you just want the position where the highest (or lowest) value is and not the actual value itself (we'll see this in a later when using the softmax activation function).

In [43]:
# Returns index of max and min values
print(f"Index where max value occurs: {x.argmax()}")
print(f"Index where min value occurs: {x.argmin()}")

Index where max value occurs: 9
Index where min value occurs: 0


More about agg [here](https://www.learnpytorch.io/00_pytorch_fundamentals/#reshaping-stacking-squeezing-and-unsqueezing)

### A.2.6 Indexing

If you've ever done indexing on Python lists or NumPy arrays, indexing in PyTorch with tensors is very similar.

In [44]:
x = torch.arange(1, 10).reshape(1, 3, 3)
x, x.shape

(tensor([[[1, 2, 3],
          [4, 5, 6],
          [7, 8, 9]]]),
 torch.Size([1, 3, 3]))

Indexing values goes outer dimension -> inner dimension (check out the square brackets).

In [45]:
# Let's index bracket by bracket
print(f"First square bracket:\n{x[0]}") 
print(f"Second square bracket: {x[0][0]}") 
print(f"Third square bracket: {x[0][0][0]}")

First square bracket:
tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])
Second square bracket: tensor([1, 2, 3])
Third square bracket: 1


You can also use `:` to specify "all values in this dimension" and then use a comma (`,`) to add another dimension.

In [48]:
# Get all values of 0th dimension and the 0 index of 1st dimension
x[:, 0]

tensor([[1, 2, 3]])

In [49]:
# Get all values of 0th & 1st dimensions but only index 1 of 2nd dimension
x[:, :, 1]

tensor([[2, 5, 8]])

In [50]:
# Get all values of the 0 dimension but only the 1 index value of the 1st and 2nd dimension
x[:, 1, 1]

tensor([5])

In [54]:
# Get index 0 of 0th and 1st dimension and all values of 2nd dimension 
x[0, 0, :] # same as x[0][0]

tensor([1, 2, 3])

For more complex data selection, such as filtering based on one or more conditions, you can use Boolean Masking-> Using a boolean tensor to select elements that meet a certain condition (e.g., x[x > 5]).

In [59]:
x>5

tensor([[[False, False, False],
         [False, False,  True],
         [ True,  True,  True]]])

In [60]:
x[x>5]

tensor([6, 7, 8, 9])

More about indexing [here](https://github.com/HassanAlgoz/B5/blob/main/W4_DL/C1_M1_Getting_Started/C1_M1_Lab_3_tensors/C1_M1_Lab_3_tensors.ipynb)

## A.3 Seeing models as computation graphs

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/appendix-a_compressed/7.webp" width="600px">

In [16]:
import torch.nn.functional as F

y = torch.tensor([1.0])  # true label
x1 = torch.tensor([1.1]) # input feature
w1 = torch.tensor([2.2]) # weight parameter
b = torch.tensor([0.0])  # bias unit

z = x1 * w1 + b          # net input
a = torch.sigmoid(z)     # activation & output

loss = F.binary_cross_entropy(a, y)
print(loss)

tensor(0.0852)


## A.4 Automatic differentiation made easy

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/appendix-a_compressed/8.webp" width="600px">

In [17]:
import torch.nn.functional as F
from torch.autograd import grad

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad=True)
b = torch.tensor([0.0], requires_grad=True)

z = x1 * w1 + b 
a = torch.sigmoid(z)

loss = F.binary_cross_entropy(a, y)

grad_L_w1 = grad(loss, w1, retain_graph=True)
grad_L_b = grad(loss, b, retain_graph=True)

print(grad_L_w1)
print(grad_L_b)

(tensor([-0.0898]),)
(tensor([-0.0817]),)


In [18]:
loss.backward()

print(w1.grad)
print(b.grad)

tensor([-0.0898])
tensor([-0.0817])


## A.5 Implementing multilayer neural networks

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/appendix-a_compressed/9.webp" width="500px">

In [19]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()

        self.layers = torch.nn.Sequential(
                
            # 1st hidden layer
            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),

            # 2nd hidden layer
            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),

            # output layer
            torch.nn.Linear(20, num_outputs),
        )

    def forward(self, x):
        logits = self.layers(x)
        return logits

In [20]:
model = NeuralNetwork(50, 3)

In [21]:
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [22]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total number of trainable model parameters:", num_params)

Total number of trainable model parameters: 2213


In [23]:
print(model.layers[0].weight)

Parameter containing:
tensor([[ 6.7373e-02,  1.2258e-01,  1.5680e-02,  ..., -9.3615e-02,
         -5.4377e-02, -1.1339e-02],
        [ 6.2112e-02,  1.7855e-02, -1.3364e-01,  ..., -1.0177e-01,
         -7.1972e-02,  3.7760e-02],
        [ 9.9113e-02, -9.5696e-03, -2.7546e-02,  ..., -4.7148e-02,
         -5.6929e-02, -9.9627e-02],
        ...,
        [ 2.0754e-02, -1.1377e-01, -5.3522e-03,  ...,  4.9732e-02,
         -2.5910e-02,  1.2745e-01],
        [-1.3523e-01,  1.9035e-02, -4.9283e-02,  ..., -5.9153e-02,
          7.3075e-05,  1.1681e-01],
        [ 3.2179e-02, -9.7925e-02, -9.8115e-02,  ..., -1.3490e-01,
          3.6057e-02, -6.5279e-02]], requires_grad=True)


In [24]:
torch.manual_seed(123)

model = NeuralNetwork(50, 3)
print(model.layers[0].weight)

Parameter containing:
tensor([[-0.0577,  0.0047, -0.0702,  ...,  0.0222,  0.1260,  0.0865],
        [ 0.0502,  0.0307,  0.0333,  ...,  0.0951,  0.1134, -0.0297],
        [ 0.1077, -0.1108,  0.0122,  ...,  0.0108, -0.1049, -0.1063],
        ...,
        [-0.0787,  0.1259,  0.0803,  ...,  0.1218,  0.1303, -0.1351],
        [ 0.1359,  0.0175, -0.0673,  ...,  0.0674,  0.0676,  0.1058],
        [ 0.0790,  0.1343, -0.0293,  ...,  0.0344, -0.0971, -0.0509]],
       requires_grad=True)


In [25]:
print(model.layers[0].weight.shape)

torch.Size([30, 50])


In [26]:
torch.manual_seed(123)

X = torch.rand((1, 50))
out = model(X)
print(out)

tensor([[-0.1262,  0.1080, -0.1792]], grad_fn=<AddmmBackward0>)


In [27]:
with torch.no_grad():
    out = model(X)
print(out)

tensor([[-0.1262,  0.1080, -0.1792]])


In [28]:
with torch.no_grad():
    out = torch.softmax(model(X), dim=1)
print(out)

tensor([[0.3113, 0.3934, 0.2952]])


## A.6 Setting up efficient data loaders

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/appendix-a_compressed/10.webp" width="600px">

In [29]:
X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])

y_train = torch.tensor([0, 0, 0, 1, 1])

In [30]:
X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6],
])

y_test = torch.tensor([0, 1])

In [31]:
from torch.utils.data import Dataset


class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels = y

    def __getitem__(self, index):
        one_x = self.features[index]
        one_y = self.labels[index]        
        return one_x, one_y

    def __len__(self):
        return self.labels.shape[0]

train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)

In [32]:
len(train_ds)

5

In [33]:
from torch.utils.data import DataLoader

torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0
)

In [34]:
test_ds = ToyDataset(X_test, y_test)

test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0
)

In [35]:
for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)

Batch 1: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])
Batch 2: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 3: tensor([[ 2.7000, -1.5000]]) tensor([1])


In [36]:
train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

In [37]:
for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)

Batch 1: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 2: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/appendix-a_compressed/11.webp" width="600px">

## A.7 A typical training loop

In [38]:
import torch.nn.functional as F


torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

num_epochs = 3

for epoch in range(num_epochs):
    
    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):

        logits = model(features)
        
        loss = F.cross_entropy(logits, labels) # Loss function
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        ### LOGGING
        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
              f" | Batch {batch_idx+1:03d}/{len(train_loader):03d}"
              f" | Train/Val Loss: {loss:.2f}")

    model.eval()
    # Optional model evaluation

Epoch: 001/003 | Batch 001/002 | Train/Val Loss: 0.75
Epoch: 001/003 | Batch 002/002 | Train/Val Loss: 0.65
Epoch: 002/003 | Batch 001/002 | Train/Val Loss: 0.44
Epoch: 002/003 | Batch 002/002 | Train/Val Loss: 0.13
Epoch: 003/003 | Batch 001/002 | Train/Val Loss: 0.03
Epoch: 003/003 | Batch 002/002 | Train/Val Loss: 0.00


In [39]:
model.eval()

with torch.no_grad():
    outputs = model(X_train)

print(outputs)

tensor([[ 2.8569, -4.1618],
        [ 2.5382, -3.7548],
        [ 2.0944, -3.1820],
        [-1.4814,  1.4816],
        [-1.7176,  1.7342]])


In [40]:
torch.set_printoptions(sci_mode=False)
probas = torch.softmax(outputs, dim=1)
print(probas)

predictions = torch.argmax(probas, dim=1)
print(predictions)

tensor([[    0.9991,     0.0009],
        [    0.9982,     0.0018],
        [    0.9949,     0.0051],
        [    0.0491,     0.9509],
        [    0.0307,     0.9693]])
tensor([0, 0, 0, 1, 1])


In [41]:
predictions = torch.argmax(outputs, dim=1)
print(predictions)

tensor([0, 0, 0, 1, 1])


In [42]:
predictions == y_train

tensor([True, True, True, True, True])

In [43]:
torch.sum(predictions == y_train)

tensor(5)

In [44]:
def compute_accuracy(model, dataloader):

    model.eval()
    correct = 0.0
    total_examples = 0
    
    for idx, (features, labels) in enumerate(dataloader):
        
        with torch.no_grad():
            logits = model(features)
        
        predictions = torch.argmax(logits, dim=1)
        compare = labels == predictions
        correct += torch.sum(compare)
        total_examples += len(compare)

    return (correct / total_examples).item()

In [45]:
compute_accuracy(model, train_loader)

1.0

In [46]:
compute_accuracy(model, test_loader)

1.0

## A.8 Saving and loading models

In [47]:
torch.save(model.state_dict(), "model.pth")

In [48]:
model = NeuralNetwork(2, 2) # needs to match the original model exactly
model.load_state_dict(torch.load("model.pth", weights_only=True))

<All keys matched successfully>

## A.9 Running tensors on GPUs (and making faster computations)

LLMs (actually all deep learning algorithms) require a lot of numerical operations.

And by default these operations are often done on a CPU (computer processing unit).

However, there's another common piece of hardware called a GPU (graphics processing unit), which is often much faster at performing the specific types of operations neural networks need (matrix multiplications) than CPUs.Your computer might have one.

If so, you should look to use it whenever you can to train neural networks because chances are it'll speed up the training time dramatically.

To check if you've got access to a Nvidia GPU, you can run `!nvidia-smi`

In [1]:
!nvidia-smi

Tue Feb 24 23:21:23 2026       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 532.09                 Driver Version: 532.09       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2070 w...  WDDM | 00000000:01:00.0 Off |                  N/A |
| N/A   57C    P8                8W /  N/A|     81MiB /  8192MiB |      3%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--


If you don't have a Nvidia GPU accessible, the above will output something like:

`NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.`

In that case, go back up and follow the install steps.

Once you've got a GPU ready to access, the next step is getting PyTorch to use for storing data (tensors) and computing on data (performing operations on tensors).

To do so, you can use the `torch.cuda` package. You can test if PyTorch has access to a GPU using `torch.cuda.is_available()`.

In [2]:
# Check for GPU
import torch
torch.cuda.is_available()

True

If the above outputs True, PyTorch can see and use the GPU, if it outputs False, it can't see the GPU and in that case, you'll have to go back through the installation steps.

Now, let's say you wanted to setup your code so it ran on CPU or the GPU if it was available.

That way, if you or someone decides to run your code, it'll work regardless of the computing device they're using.

Let's create a device variable to store what kind of device is available.

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


If the above output "cuda" it means we can set all of our PyTorch code to use the available CUDA device (a GPU) and if it output "cpu", our PyTorch code will stick with the CPU.

> Note: In PyTorch, it's best practice to write device agnostic code. This means code that'll run on CPU (always available) or GPU (if available).

If you want to do faster computing you can use a GPU but if you want to do much faster computing, you can use multiple GPUs.

You can count the number of GPUs PyTorch has access to using `torch.cuda.device_count()`.

In [4]:
torch.cuda.device_count()

1

### Getting PyTorch to run on Apple Silicon
In order to run PyTorch on Apple's M1/M2/M3 GPUs you can use the torch.backends.mps module.

Be sure that the versions of the macOS and Pytorch are updated.

You can test if PyTorch has access to a GPU using torch.backends.mps.is_available()

In [5]:
import torch
torch.backends.mps.is_available() # Note this will print false if you're not running on a Mac

False

In [6]:
# Set device type -- uncomment only if you are mac
#device = "mps" if torch.backends.mps.is_available() else "cpu"

As before, if the above output "mps" it means we can set all of our PyTorch code to use the available Apple Silicon GPU.

In [10]:
#  device agnostic code 
if torch.cuda.is_available():
    device = "cuda" # Use NVIDIA GPU (if available)
elif torch.backends.mps.is_available():
    device = "mps" # Use Apple Silicon GPU (if available)
else:
    device = "cpu" # Default to CPU if no GPU is available
print(device)

cuda


To train LLMs, we need the massive parallelism of GPUs.  matrix multiplication is massively accelerated on CUDA cores.

Moving our tensors to the GPU is straightforward. PyTorch allows moving tensors to GPU using `.to(device)`.

In [49]:
x = torch.randn(1000, 1000)
W = torch.randn(1000, 1000)

y = x @ W
print("Computation complete on:", y.device)

Computation complete on: cpu


In [50]:
#torch.cuda.empty_cache()

x = torch.randn(1000, 1000).to(device)
W = torch.randn(1000, 1000).to(device)

y = x @ W
print("Computation complete on:", y.device)

Using device: cuda
Computation complete on: cuda:0
